# Substituir mapbiomas pelas áreas agropecuárias do projeto?

In [1]:
# Importação das bibliotecas necessárias
import pandas as pd

from bs4 import BeautifulSoup
import requests

In [2]:
dataset_name = 'mapbiomas_cobertura'
raiz = '../databases'

### Bronze

In [3]:
# Requisição para a URL
response = requests.get('https://brasil.mapbiomas.org/estatisticas/')

# Criação do objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Listagem de todos os links na página
links = soup.find_all('a')

# Filtro para apenas os links de download
download_links = [link.get('href') for link in links if 'download' in link.get('href')]

# Seleção do segundo link de download
link_tabela_biomas_munic = download_links[1]

# Salvamento do arquivo em disco
filename = f'{raiz}/bronze/mapbiomas_{link_tabela_biomas_munic.split("/")[-1]}'

with open(filename, 'wb') as arquivo:
    arquivo.write(requests.get(link_tabela_biomas_munic).content)

In [4]:
# coleção 9
link_tabela_biomas_munic

'https://storage.googleapis.com/mapbiomas-public/initiatives/brasil/collection_9/downloads/mapbiomas_brazil_urban_v2.xlsx'

### Fase Prata

|---------------------------------------------------------------------|
|                              COLEÇÃO 8 - CLASSES                              |
|-------------------------------------|---------------------------------|-------|
| Categoria                           | Subcategoria                    | ID    |
|-------------------------------------|---------------------------------|-------|
| 3.1. Pastagem                       |                                 | 15    |
| 3.2. Agricultura                    |                                 | 18    |
|                                     | 3.2.1. Lavoura Temporária      | 19    |
|                                     |     3.2.1.1. Soja             | 39    |
|                                     |     3.2.1.2. Cana             | 20    |
|                                     |     3.2.1.3. Arroz (beta)    | 40    |
|                                     |     3.2.1.4. Algodão (beta)  | 62    |
|                                     |     3.2.1.5. Outras Lavouras Temporárias | 41 |
|                                     | 3.2.2. Lavoura Perene          | 36    |
|                                     |     3.2.2.1. Café             | 46    |
|                                     |     3.2.2.2. Citrus           | 47    |
|                                     |     3.2.2.3. Dendê           | 35    |
|                                     |     3.2.1.3. Outras Lavouras Perenes | 48 |
| 3.4. Mosaico de Usos                |                                 | 21    |


Descrição dos códigos: https://brasil.mapbiomas.org/wp-content/uploads/sites/4/2023/08/Legenda-Colecao-8-LEGEND-CODE.pdf

In [5]:
# # Requisição para a URL
# response = requests.get('https://brasil.mapbiomas.org/estatisticas/')

# # Criação do objeto BeautifulSoup
# soup = BeautifulSoup(response.text, 'html.parser')

# # Listagem de todos os links na página
# links = soup.find_all('a')

# # Filtro para apenas os links de download
# download_links = [link.get('href') for link in links if 'download' in link.get('href')]

# # Seleção do segundo link de download
# tabela_biomas_munic = download_links[1].split("/")[-1]

In [6]:
dados_mapbiomas = pd.read_excel(filename, sheet_name=1)
dados_mapbiomas.head()

,territory_level_1,territory_level_2,territory_level_3,territory_level_4,geocode,category_id,version,class_level_0,class_level_1,class_level_2,...,y2014,y2015,y2016,y2017,y2018,y2019,y2020,y2021,y2022,y2023
0,Brasil,Rondônia,RO,Alta Floresta D'Oeste,1100015,3,0-24.1,Natural,2. Setor Censitário Urbano,2.2 Floresta,...,28.000528,27.299466,24.148888,23.011576,22.926614,25.026916,25.464394,25.026403,24.064084,12.163617
1,Brasil,Rondônia,RO,Alta Floresta D'Oeste,1100015,4,0-24.1,Natural,2. Setor Censitário Urbano,2.2 Floresta,...,33.344362,32.906713,29.493514,26.080369,26.167930,27.568203,29.318535,24.767613,19.691516,18.378733
2,Brasil,Rondônia,RO,Alta Floresta D'Oeste,1100015,6,0-24.1,Natural,2. Setor Censitário Urbano,2.2 Floresta,...,5.838684,5.925446,6.012585,5.751168,6.012962,7.581419,7.842841,8.278514,7.145710,3.576459
3,Brasil,Rondônia,RO,Alta Floresta D'Oeste,1100015,11,0-24.1,Natural,2. Setor Censitário Urbano,2.3 Formação Natural não Florestal,...,25.108113,25.983683,24.148841,23.885536,23.972677,24.146184,26.674681,25.279352,23.711992,20.043466
4,Brasil,Rondônia,RO,Alta Floresta D'Oeste,1100015,12,0-24.1,Natural,2. Setor Censitário Urbano,2.3 Formação Natural não Florestal,...,13.626591,12.140307,11.089694,9.692053,9.607191,9.170387,11.523862,10.130030,5.155948,2.099676


In [33]:
# dados_mapbiomas_limpo = (
#     dados_mapbiomas.loc[
#         dados_mapbiomas.category_id.isin((15, 18)), 
#         [col for col in dados_mapbiomas.columns.tolist() if col in ('geocode', 'territory_level_3', 'class_level_3') or col.startswith("y")]
#     ]
#         .rename(columns={"geocode": "geocodigo", "class_level_3": "atividade", "territory_level_3": "uf"})
#         .rename(columns=lambda col: col.replace("y", ""))
#         # .assign(
#         #     cd_uf = lambda df: df.geocodigo.astype(str).str.extract(r"^(\d{2})")
#         # )
#         .set_index(["uf", "geocodigo", "atividade"])
#         .groupby(['uf', "atividade"]).sum()
# )

# dados_mapbiomas_limpo.sample(3)

ano_inicial = 1990

silver_df = (
    dados_mapbiomas.loc[
        dados_mapbiomas.category_id.isin((15, 18)), 
        [col for col in dados_mapbiomas.columns.tolist() if col in ('geocode', 'territory_level_3', 'class_level_3') or col.startswith("y")]
    ]
        .rename(columns={"geocode": "geocodigo", "class_level_3": "fonte", "territory_level_3": "uf"})
        .rename(columns=lambda col: col.replace("y", ""))
        .assign(
            uf = lambda df: df.geocodigo.astype(str).str.extract(r"^(\d{2})"),
        )
        .set_index(["uf", "geocodigo", "fonte"])
        .rename(columns=int)
        .groupby(['uf', "fonte"])
        .sum()
        .stack()
        .rename_axis(['uf', "fonte", 'ano'])
        .reorder_levels([0, 2, 1])
        .sort_index()
        .rename("area_ha")
        .loc[lambda df: df.index.get_level_values("ano") >= ano_inicial]        
)

silver_df.to_frame().to_parquet(f'{raiz}/prata_{dataset_name}_areaclass_agropecuaria.parquet')

silver_df #.sample(3)

uf  ano   fonte            
11  1990  2.4.1 Pastagem       48186.169012
          2.4.2 Agricultura      991.477917
    1991  2.4.1 Pastagem       48422.882890
          2.4.2 Agricultura     1016.371671
    1992  2.4.1 Pastagem       49881.081319
                                   ...     
53  2021  2.4.2 Agricultura     3008.645836
    2022  2.4.1 Pastagem        8275.245751
          2.4.2 Agricultura     3014.325142
    2023  2.4.1 Pastagem        7059.494320
          2.4.2 Agricultura     3014.583309
Name: area_ha, Length: 1836, dtype: float64

In [8]:
# dados_mapbiomas_limpo.to_excel(f'{raiz}/bronze/mapbiomas_areas_agropecuarias_uf.xlsx')

In [9]:
# anos = (    
#     pd.read_excel(
#         filename,
#         sheet_name = 1,
#         usecols=lambda x: isinstance(x, int)
#     )
#         # .columns
#         # .drop(range(1985, 2000))
#         # .tolist()
#         # )
# )

# anos

In [10]:
# df = (pd.read_excel(f'{raiz}/bronze_{link_tabela_biomas_munic.split("/")[-1]}',
#                       sheet_name = 1)
#     .drop(range(1985, 2000), axis = 'columns')
#     .query('class_id in (15, 18, 19, 39, 20, 40, 62, 41, 36, 46, 47, 35, 48, 21)')
#     .assign(**{
#         'uf': lambda df: (df['geocode']// 100000)
#         }
#             )
#     .melt(
#         id_vars=['uf', 'geocode', 'class_id'],
#         value_vars=anos,
#         var_name='ano',
#         value_name='area_ha'
#         )
#     .dropna(subset='area_ha')
#     .convert_dtypes()
#     .rename(columns=
#             {
#                 'geocode':'geocodigo',
#                 'class_id':'classe_solo'
#              })
#     .set_index(['uf', 'geocodigo', 'ano', 'classe_solo'])
#     )

In [11]:
# (df.reset_index()
#  .assign(**{
#           'fonte': 'Deposição Atmosférica'
#           })
#  .groupby(['uf', 'geocodigo', 'ano', 'fonte'])
#  .agg({'area_ha':sum})
#  .to_parquet(f'{raiz}/prata_{dataset_name}_areatotal_agropecuaria.parquet')
#  )

### Fase Ouro

Área Destinada a Agropecuária (ha)
Elementos (kg/ha ano)

Cálculo: Cobertura Solo Municipal * (Parâmetro_X/1000)

UF e Brasil = mil t

NPK oriundo de deposição atmosférica incorporado ao solo (t)

In [34]:
parametro_N = 5.5
parametro_P = 0.3
parametro_K = 3.1

In [36]:
df = (
    # pd.read_parquet(f'{raiz}/prata_{dataset_name}_areatotal_agropecuaria.parquet')
    pd.read_parquet(f'{raiz}/prata_{dataset_name}_areaclass_agropecuaria.parquet')
        .reset_index()
        .assign(**{
            'fonte': 'Deposição Atmosférica',
            'N': lambda df: df['area_ha']*(parametro_N),
            'P': lambda df: df['area_ha']*(parametro_P),
            'K': lambda df: df['area_ha']*(parametro_K)
        }) 
        .drop('area_ha', axis="columns")
        .groupby(["uf", "ano", "fonte"]).agg('sum')
        .reset_index()
        .melt(
            id_vars=['uf', 'ano', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor'
        )
        .set_index(['uf', 'ano', 'fonte', 'elemento'])
        .sort_index()
        .squeeze()
)

df.to_csv(f'{raiz}/ouro_npk_deposicao_atmosferica.csv')
df

uf  ano   fonte                  elemento
11  1990  Deposição Atmosférica  K           152450.705481
                                 N           270477.058112
                                 P            14753.294079
    1991  Deposição Atmosférica  K           153261.689139
                                 N           271915.900085
                                                 ...      
53  2022  Deposição Atmosférica  N            62092.639910
                                 P             3386.871268
    2023  Deposição Atmosférica  K            31229.640652
                                 N            55407.426964
                                 P             3022.223289
Name: valor, Length: 2754, dtype: float64